In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso
from scipy.fftpack import dct,idct

In [0]:
data = pd.read_csv('samples.csv',skiprows=[1])


In [0]:
# Extracting the seperate column values

x = data["'Elapsed time'"]
x = x.values
y = data["'V5'"]
y = y.values

y_dct = dct(y,norm='ortho')               #dct matrix of the signal
plt.subplot(211)
plt.plot(x,y)
plt.xlabel('Samples')
plt.ylabel('Amplitude')
plt.title('Original Signal')
plt.subplot(212)
plt.plot(np.linspace(0,1000,1000),y_dct[0:1000])
plt.show()


In [0]:
# sparse matrix

sparsity = 0.4
m = int(sparsity*len(x))

ri = np.random.choice(3600, m, replace=False)     #random samples of indices
ri.sort()
xc = x[ri]
yc = y[ri]
yc_dft = dct(yc,norm='ortho')
plt.subplot(211)
plt.plot(xc,yc)
plt.subplot(212)
plt.plot(np.linspace(0,1,m),yc_dft)
plt.show()

 The reconstruction part is found to be faulty but it detects all the peaks in the ecg signal.

In [0]:
# reconstruction of sparse ecg signal

A = idct(np.identity(len(x)),norm='ortho',axis=0)
A = A[ri]

In [0]:
L1_min = Lasso(alpha=0.0001,normalize=True,max_iter=1e5)
for i in range(10):
  L1_min.fit(A,yc)

In [0]:
#DCT of the reconstructed signal

Rcon_dct = L1_min.coef_
#Rcon_dct = np.squeeze(Rcon_dct)
#Rcon_dct[200:(len(Rcon_dct)-1)] = 0
Rcon_sig = idct(Rcon_dct, norm='ortho', axis=0)

In [0]:
plt.subplot(421)
plt.plot(x,Rcon_sig)
plt.title('Reconstructed signal')
plt.subplot(422)
plt.plot(np.linspace(0,1000,1000),Rcon_dct[0:1000])
plt.title('Spectrum of reconstructed signal')
plt.subplot(423)
plt.plot(x,y)
plt.title('Original Signal')
plt.subplot(424)
plt.plot(np.linspace(0,100,100),y_dct[0:100])
plt.title('Spectrum of original signal')
plt.show()
print('Error=',np.mean(np.abs(y-Rcon_sig)))